In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc,html,Input, Output,dash_table
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


In [2]:
seoul_bike = pd.read_parquet('D:/git_local_repository/data_for_project/seoul_bike/seoul_bike_2021.parquet.gzip')

### 아래부터 개발용

In [3]:
# app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY])
# station = pd.read_csv('assets/seoul_bike_station_01_12.csv',
#                       encoding='CP949', index_col=0)  # 따릉이 대여소 정보
# station_id_dict = station[['id', '주소2']].to_dict('records')
# value = '207'
# ids_dropdown = {station_id_dict[i].get(
#     'id'): f" {station_id_dict[i].get('id')} : {station_id_dict[i].get('주소2')}" for i in range(len(station_id_dict))}
# val = ids_dropdown.get(int(value))


# # {"102 : 망원역 1번출구 앞" : "102"}
# ids_dropdown = {station_id_dict[i].get(
#     'id'): f" {station_id_dict[i].get('id')} : {station_id_dict[i].get('주소2')}" for i in range(len(station_id_dict))}

# app.layout = dbc.Col([dcc.Dropdown(ids_dropdown, "207", placeholder='search id or name', id='test_dropdown'),  # disabled=True ,
#                       html.H1(id='output_place_name')
#                       ])


# @app.callback(
#     Output(component_id='output_place_name', component_property='children'),
#     Input('test_dropdown', 'value')
# )
# def search_id(value):
#     val = ids_dropdown.get(int(value))
#     return val


# app.run_server(debug=True, mode='external', height=500, width='80%')


### 위부터 개발용

In [130]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SANDSTONE])
server = app.server

# 필수 정보 불러오기
station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)  # 따릉이 대여소 정보
near_sub = pd.read_csv(
    "assets/near_sub_station.csv", encoding="CP949", index_col=0
)  # 지하철역 인근 따릉이 대여소 정보


def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m


# dropdown 용도
# {"102 : 망원역 1번출구 앞" : "102"}
station_id_dict = station[["id", "주소2"]].to_dict("records")
ids_dropdown = {
    station_id_dict[i].get(
        "id"
    ): f" {station_id_dict[i].get('id')} : {station_id_dict[i].get('주소2')}"
    for i in range(len(station_id_dict))
}


app.layout = dbc.Container(
    [
        dbc.Row([], style={"height": "50px"}),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.Col(
                            [
                                dcc.Dropdown(
                                    ids_dropdown,
                                    "754",
                                    placeholder="search id or name",
                                    id="test_dropdown",
                                    style={
                                        "border": "#fff",
                                        "width": "90%",
                                        "background-color" : "#fff"
                                    },
                                    clearable=False,
                                )
                            ],
                            md=12,
                            class_name="shadow-sm h4 py-3",
                        ),
                        dcc.Graph(id="searching_station", className="col-12",style={"background-color" : "#fff"}),
                    ],
                    lg=8,
                    class_name="shadow-sm p-3",
                ),
                dbc.Col(
                    [
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        html.Span(["출발 :"]), 
                                        dcc.Dropdown(
                                            ["대여소", "역"],
                                            value="대여소",
                                            style={
                                                "border-color": "#fff",
                                                "width": "70%",
                                            },
                                            id="station_or_sub",
                                            clearable=False,
                                        ),
                                    ],
                                    
                                    style={"align-items": "center", "display": "flex",},
                                    class_name="p-0 mx-auto",
                                ),
                                dbc.Col(
                                    [
                                        "지하철역 개수 :",
                                        dcc.Dropdown(
                                            list(range(1, 15)),
                                            value=5,
                                            id="rank_substation",
                                            style={"border-color": "#fff"},
                                            clearable=False,
                                        ),
                                    ],

                                    class_name="p-0 mx-auto",
                                    style={"align-items": "center",
                                                "display": "flex",}
                                ),
                            ],
                            style={'font-size' : '20px','margin-top' : '15px'},
                            class_name='p-2'
                        ),
                        dbc.Col(lg=12, style={"height": "50px"}),
                        dbc.Col(
                            [html.H3("대여소 별 예상 도착시간")],
                            lg=12,
                            style={"text-align": "center", 'margin-bottom' : '5px' },
                        ),
                        dbc.Col(id="table-3", lg=12, class_name="overflow-auto",style={'height' : '450px'}),
                    ],
                    class_name="shadow-sm p-2",
                    style={"background-color" : "#fff"},
                    lg=4,
                ),
            ],
            align="top",
            class_name="shadow",
            style={"background-color" : "#fff"}
        ),
    ],
    
)


# @app.callback(
#     Output(component_id="station_name_output", component_property="children"),
#     Input("test_dropdown", "value"),  # Input('location','value'),
# )
# def searching_st_name(value):
#     val = ids_dropdown.get(int(value))
#     return val


@app.callback(
    # Output(component_id="table-1", component_property="children"),
    # Output(component_id="table-2", component_property="children"),
    Output(component_id="table-3", component_property="children"),
    Output(component_id="searching_station", component_property="figure"),
    Input("test_dropdown", "value"),  # Input('location','value'),
    Input("rank_substation", "value"),  # Input('location','value'),
    Input("station_or_sub", "value"),  # Input('location','value'),
)
def finding_substation(stat_id, rank_substation, filter_start, include_st=300):
    """
    stat_id : 기준 대여소
    incldue_st : 기준 대여소 600미터 내 대여소 검색
    n : 대여소간 이동횟수가 최소 10회 이상인 경우만 sorting, ex) 754 to 다른대여소 중 10회 이상 이동이 있는 경우만 계산에 포함

    return : 1 to 2 이동 정보, 2 to 1 이동 정보, 시간정보
    """

    ##################################
    stat_id = int(stat_id)
    rank_substation = int(rank_substation)

    # 필수 정보 불러오기
    station = pd.read_csv(
        "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
    )  # 따릉이 대여소 정보
    near_sub = pd.read_csv(
        "assets/near_sub_station.csv", encoding="CP949", index_col=0
    )  # 지하철역 인근 따릉이 대여소 정보

    # stat_id 기준으로 반경 600m 내 대여소 찾기
    distance = haversine_np(
        station.loc[station["id"] == stat_id, "경도"].values,
        station.loc[station["id"] == stat_id, "위도"].values,
        station["경도"].values,
        station["위도"].values,
    )
    a = station.copy()
    a["distance"] = distance
    a.dropna(inplace=True)
    # 대여소 검색 결과
    ids = (
        a.query("distance < @include_st").sort_values(by="distance")["id"].tolist()
    )  

    # 검색된 대여소 id 출력
    print(f"{stat_id} 대여소 기준 {include_st}m 반경 내 대여소 : {ids}")

    # seoul_bike에서 해당 대여소 자료만 추출(자료 검색을 빠르게 하기 위함!)
    reduce_case = []
    for st_id in ids:
        BM = (seoul_bike["st_id1"] == st_id) | (seoul_bike["st_id2"] == st_id)
        sort_754 = seoul_bike[BM]

        # 대여소 이동이 없는 경우 제거 ex) 754 => 754
        minus = sort_754.query("st_id1 == @st_id & st_id2 == @st_id").index
        sorting = sort_754[~sort_754.index.isin(minus)]
        reduce_case.append(sorting)

    # 이제는 원본 자료가 아닌 sorted_station을 가지고 이동량 계산
    sorted_station = pd.concat(reduce_case, axis=0)

    ##################################

    # 개별 대여소별 이동량 계산 후 저장
    result = []
    for st_id in ids:

        # ex) 기준대여소 to 다른 대여소 (1to2)
        total_num_left = sorted_station[sorted_station["st_id1"] == st_id][
            "st_id2"
        ].value_counts()  # 기준 대여소에서 빌리면 주로 향하는 목적지는?

        # ex) 다른 대여소 to 기준대여소 (2to1)
        total_num_right = sorted_station[sorted_station["st_id2"] == st_id][
            "st_id1"
        ].value_counts()  # 어떤 대여소에서 기준 대여소로 이동할까?

        # 1to2, 2to1 합치기
        combine_values = pd.concat([total_num_left, total_num_right], axis=1)
        combine_values.fillna((1), inplace=True)
        combine_values.rename(columns=dict(st_id2="1to2", st_id1="2to1"), inplace=True)
        result.append(combine_values)

    # 계산 결과 종합
    result_concat_proto = pd.concat(result, axis=0)
    result_concat = (
        result_concat_proto.reset_index().groupby("index")[["1to2", "2to1"]].sum()
    )

    # 역 주변 대여소가 검색될경우 해당 대여소는 제거하기 | 316 대여소는 종각역과 관련됐는데, 종각과 관련된 대여소는 제거했다.
    try:
        filter_sub = near_sub.query("bike_id == @stat_id")["역사명"].iloc[0]
        filter_sub_2 = near_sub.query("역사명 ==@filter_sub")["bike_id"].values
        result_concat = result_concat[~result_concat.index.isin(filter_sub_2)]
    except:
        pass

    # 필요 정보 추가
    # result_concat['rate'] = round((result_concat["1to2"] / (result_concat["1to2"]+result_concat["2to1"])),4) # 비율
    
    # 대여소 별 총 이동 횟수
    result_concat["total"] = abs(
        (result_concat["1to2"] + result_concat["2to1"])
    )  

    # result_concat['category'] = pd.cut(
    #                                     result_concat['rate'],
    #                                     bins=[0,0.45,0.55,result_concat['rate'].max()],
    #                                     labels=["to", "sim" ,"from"]
    #                                 )
    # to : to 754로 기억하자. 해당대여소가 754로 오는 경우가 많다. 0~0.45 => 1to2가 2to1보다 작다.
    # sim : 0.45~0.55 => 1to2와 2to1이 비슷하다. 즉 왕래가 비슷한 대여소이다.
    # from : from 754로 기억하자. 754 대여소가 해당 대여소로 향하는 경우가 많음. 0.55~1 => 1to2가 2to1보다 많다.

    # 지하철역 인근 따릉이 대여소 정보와 종합
    sub_bike = pd.merge(
        result_concat,
        near_sub,
        how="left",
        left_on=result_concat.index,
        right_on="bike_id",
    )

    # 이용률 top 20 대여소 선정
    sorted_sub = sub_bike.dropna(subset=["역사명"]).sort_values(
        by="total", ascending=False
    )[:20]

    # 기준 대여소와 역근처 대여소 직선 거리계산
    station_lat_lon = station[station["id"].isin(sorted_sub["bike_id"])][
        ["id", "주소2", "위도", "경도"]
    ]
    dist_to_station = haversine_np(
        station.loc[station["id"] == stat_id, "경도"].values,
        station.loc[station["id"] == stat_id, "위도"].values,
        station_lat_lon["경도"].values,
        station_lat_lon["위도"].values,
    )
    station_lat_lon["distance"] = dist_to_station

    # 거리정보 종합
    sorted_sub = pd.merge(
        sorted_sub, station_lat_lon, left_on="bike_id", right_on="id"
    ).drop(columns=["id"])

    # 대여소별 예상 도착시간 계산
    result_station = []
    for station_id in sorted_sub["bike_id"]:

        # 대여소 기준 해당 역으로 가는 시간
        BM = sorted_station["st_id2"] == station_id
        st_id1_time = (
            sorted_station[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        mean_id1 = round(st_id1_time.index[:3].values.mean(), 1)

        # 해당 역에서 대여소로 가는 시간
        BM = sorted_station["st_id1"] == station_id
        st_id2_time = (
            sorted_station[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        mean_id2 = round(st_id2_time.index[:3].values.mean(), 1)

        # st_id1_time.plot.bar()
        result_station.append([mean_id1, mean_id2])

    # 예상시간정보 종합(대여소: 대여소에서 출발)
    est_time = pd.DataFrame(result_station, columns=["대여소", "역"])

    # return 자료 생성
    nearest_sub = pd.concat([sorted_sub.reset_index(drop=True), est_time], axis=1)

    if filter_start == "대여소":
        sub_sorted_station = (
            nearest_sub.groupby("역사명")["1to2"]
            .sum()
            .sort_values(ascending=False)
            .reset_index(drop=False)[:rank_substation]
        )  # 대여소 별 이동량 종합
        counts = "1to2"

    elif filter_start == "역":
        sub_sorted_station = (
            nearest_sub.groupby("역사명")["2to1"]
            .sum()
            .sort_values(ascending=False)
            .reset_index(drop=False)[:rank_substation]
        )
        counts = "2to1"

    # 역 이름 추출
    name_sub = sub_sorted_station["역사명"].tolist()

    # 대여소 예상시간 테이블 만들기
    nearest_sub_sorted = (
        nearest_sub[["역사명", "bike_id", counts, f"{filter_start}"]]
        .sort_values(by=f"{filter_start}")
        .query("역사명 == @name_sub")
        .reset_index(drop=True)
    )
    nearest_sub_sorted.columns = ["역사명", "대여소_ID", "대여기록", "예상시간"]
    nearest_sub_sorted = nearest_sub_sorted.query("대여기록 > 50").copy()

    nearest_sub_sorted["예상시간"] = nearest_sub_sorted["예상시간"].apply(
        lambda x: str(int(x)) + "분"
    )
    nearest_sub_sorted["대여기록"] = nearest_sub_sorted["대여기록"].apply(
        lambda x: str(int(x)) + "건"
    )
    nearest_sub_sorted_table = dbc.Table.from_dataframe(
        nearest_sub_sorted, striped=True, hover=True, style={"text-align": "center"},class_name="overflow-auto"
    )

    #####

    # figure 만들기
    fig = px.scatter_mapbox(
        nearest_sub.query("역사명 == @name_sub"),
        lat="위도",
        lon="경도",
        hover_name="주소2",
        hover_data=["1to2", "2to1", "bike_id", f"{filter_start}"],
        color="역사명",
        opacity=0.5,
        template="seaborn",
        mapbox_style="carto-positron",
        # size='total', size_max=20,
        zoom=12,
        height=500,
    )
    fig.for_each_trace(lambda t: t.update(name="<b>" + t.name + "</b>"))
    fig.update_traces(marker={"size": 15})

    # 해당 따릉이 대여소 색 표시
    fig_3 = go.Figure(
        go.Scattermapbox(
            name="대여소",
            lat=station.query("id==@ids")["위도"],
            lon=station.query("id==@ids")["경도"],
            mode="markers+text",
            marker={"size": 10, "color": "cyan", "opacity": 0.4},
            text=station.query("id==@ids")["주소2"].values,
        )
    )

    fig.add_trace(fig_3.data[0])
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bgcolor="rgba(223, 235, 223, 0.3)",
        ),
    )

    return nearest_sub_sorted_table, fig
    # return one_to_two_table, two_to_one_table, nearest_sub_sorted_table, fig


app.run_server(debug=True, mode="external", height=1000, width="80%")



Dash app running on http://127.0.0.1:8050/
754 대여소 기준 300m 반경 내 대여소 : [754, 4526]
754 대여소 기준 300m 반경 내 대여소 : [754, 4526]


### Check 2

In [ ]:
    # nearest_sub_dt = dash_table.DataTable(
    #     data=data,
    #     columns=columns,
    #     style_cell_conditional=[
    #         {"if": {"column_id": c}, "textAlign": "left"} for c in ["Date", "Region"]
    #     ],
    #     style_data={"color": "black", "backgroundColor": "white"},
    #     style_data_conditional=[
    #         {"if": {"row_index": "odd"}, "backgroundColor": "rgb(220, 220, 220)",}
    #     ],
    #     style_header={
    #         "backgroundColor": "rgb(210, 210, 210)",
    #         "color": "black",
    #         "fontWeight": "bold",
    #     },
    # )

In [ ]:
    # nearest_sub_sorted = (
    #     nearest_sub[["역사명", "bike_id", "대여소_to_역(분)", "역_to_대여소(분)"]]
    #     .sort_values(by="대여소_to_역(분)")
    #     .iloc[:10]
    # )
    # nearest_sub_sorted["대여소_to_역(분)"] = (
    #     nearest_sub_sorted["대여소_to_역(분)"].astype("str") + "분"
    # )
    # nearest_sub_sorted["역_to_대여소(분)"] = (
    #     nearest_sub_sorted["역_to_대여소(분)"].astype("str") + "분"
    # )

    # nearest_sub_sorted.columns = ["역사명", "대여소_ID", "대여소 → 역", "역 → 대여소"]
    # nearest_sub_sorted_table = dbc.Table.from_dataframe(
    #     nearest_sub_sorted, striped=True, hover=True, style={"text-align": "center"}
    # )

    # one_to_two.columns = ["역사명", "대여기록"]
    # one_to_two_table = dbc.Table.from_dataframe(
    #     one_to_two, striped=True, hover=True, style={"text-align": "center"}
    # )

    # two_to_one.columns = ["역사명", "대여기록"]
    # two_to_one_table = dbc.Table.from_dataframe(
    #     two_to_one, striped=True, hover=True, style={"text-align": "center"}
    # )

In [ ]:
                                        # dbc.Col(
                                        #     [html.H3("대여소 → 역")],
                                        #     lg=6,
                                        #     style={"text-align": "center"},
                                        # ),
                                        # dbc.Col(
                                        #     [html.H3("역 → 대여소")],
                                        #     lg=6,
                                        #     style={"text-align": "center"},
                                        # ),
                                        # dbc.Col(id="table-1", lg=6),
                                        # dbc.Col(id="table-2", lg=6),